In [1]:
import pandas as pd
import tweepy
import requests
import re
import emoji
from access_token import tokens

In [2]:
auth = tweepy.OAuthHandler(tokens['api_token'], tokens['api_token_secret'])
auth.set_access_token(tokens['access_token'], tokens['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
# print(emoji_pattern.sub(r'', text)) # no emoji

In [4]:
# Collect tweets
query = "#trump" + " -filter:retweets"
cutoff_date = "2022-03-19"
tweets = tweepy.Cursor(api.search_tweets, q=query, lang="en", until=cutoff_date).items(200)

tweets_list = [[tweet.created_at, tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]

tweets_df = pd.DataFrame(data=tweets_list, columns=['date', 'user', 'location', 'text'])

In [5]:
tweets_df

date            user       location  \
0   2022-03-18 23:59:04+00:00    sosajoejr559     Hanford CA   
1   2022-03-18 23:59:01+00:00         W_Mead_                  
2   2022-03-18 23:58:57+00:00     LegalLining  The Big Apple   
3   2022-03-18 23:58:33+00:00    RedCacheONLY                  
4   2022-03-18 23:58:00+00:00       Theboveed                  
..                        ...             ...            ...   
195 2022-03-18 23:01:29+00:00    DorothyBeach      Ohio, USA   
196 2022-03-18 23:00:46+00:00    randomtrump1                  
197 2022-03-18 23:00:34+00:00         cozharz           Troy   
198 2022-03-18 23:00:07+00:00  DemActionToday                  
199 2022-03-18 23:00:00+00:00       c_baschan                  

                                                  text  
0    45th President Donald J. Trump to Hold a Rally...  
1    @MarshaBlackburn #trump and the @GOP already t...  
2    Russia prepared Trump to be Putin's Puppet for...  
3    Alex Jones on why Full Send Podcast Trump Inte...  
4    Donald Trump wants his supporters to carry Tru...  
..                                                 ...  
195  @JudiciaryDems An honor that was not given to ...  
196  Random Trump https://t.co/wIrKt8Vmsq - #random...  
197  This #atheist murderous KGB #dictator quoted #...  
198  @mazemoore #PutinsPuppet https://t.co/QNNLLMpQ...  
199  Critical Race Theory explained by John Oliver ...  

[200 rows x 4 columns]

In [6]:
tweets_df.to_csv ('test.csv', quotechar='"', encoding='utf8', index = False, header=True)

# Data Cleaning

In [9]:
# # Uncomment the line below if loading from previously saved CSV
# tweets_df = pd.read_csv('test.csv', quotechar='"', encoding='utf8')

# Remove punctuation
tweets_df['text'] = tweets_df['text'].map(lambda x: re.sub('[,\\.!?]', ' ', x))

# Remove unnecessary line breaks
tweets_df['text'] = tweets_df['text'].map(lambda x: re.sub(r"\n", '', x))

# Convert the titles to lowercase
tweets_df['text'] = tweets_df['text'].map(lambda x: x.lower())

# Remove emojis
tweets_df['text'] = tweets_df['text'].map(lambda x: emoji.demojize(x))

# # Print out the first rows 
# tweets_df['text']

0      45th president donald j  trump to hold a rally...
1      @marshablackburn #trump and the @gop already t...
2      russia prepared trump to be putin's puppet for...
3      alex jones on why full send podcast trump inte...
4      donald trump wants his supporters to carry tru...
                             ...                        
195    @judiciarydems an honor that was not given to ...
196    random trump https://t co/wirkt8vmsq - #random...
197    this #atheist murderous kgb #dictator quoted #...
198    @mazemoore #putinspuppet https://t co/qnnllmpq...
199    critical race theory explained by john oliver ...
Name: text, Length: 200, dtype: object

In [10]:
# # Removing duplicate tweets?
# unique_tweets = list(set(list(tweets_df['text_processed'].values)))
# unique_tweets = [t for t in unique_tweets if t]

# print(unique_tweets)

In [11]:
tweets_df.to_csv ('test_emoji_removed.csv', quotechar='"', encoding='utf8', index = False, header=True)